<a href="https://colab.research.google.com/github/senthilv83/Data-Science-Challenges/blob/main/Topic_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving reviews - reviews.csv to reviews - reviews.csv


In [ ]:
import pandas as pd

In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['reviews - reviews.csv']))

In [ ]:
#import nltk; nltk.download('stopwords')
import spacy.cli
spacy.cli.download("en_core_web_lg")
import en_core_web_lg


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 4.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=8976d0b9b2274ff5ac8e0248e8a7f44994c8284a978f34e4fdfa5409008b573a
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=d1e119108138f7c9042d2fd5549fca929c15f00aea5e7fc89aa37cca98d380dc
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built pyLDAvis sklearn


In [ ]:
!pip install pyLDAvis.gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement pyLDAvis.gensim (from versions: none)
ERROR: No matching distribution found for pyLDAvis.gensim


In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline



In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
data = df.pros.values.tolist() + df.pros.values.tolist()

# Remove Emails
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub(r'\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['Excellent work atmosphere. Ample opportunities for growth.']


In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['excellent', 'work', 'atmosphere', 'ample', 'opportunities', 'for', 'growth']]


In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['excellent', 'work', 'atmosphere', 'ample', 'opportunities', 'for', 'growth']


In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en_core_web_lg' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['excellent', 'work', 'atmosphere', 'ample', 'opportunity', 'growth']]


In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [ ]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('ample', 1),
  ('atmosphere', 1),
  ('excellent', 1),
  ('growth', 1),
  ('opportunity', 1),
  ('work', 1)]]

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

In [ ]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.544*"work" + 0.103*"home" + 0.066*"staff" + 0.057*"hour" + '
  '0.053*"flexible" + 0.027*"ability" + 0.020*"variety" + 0.012*"corporate" + '
  '0.009*"working" + 0.007*"commute"'),
 (1,
  '0.085*"large" + 0.060*"customer" + 0.048*"base" + 0.046*"s" + '
  '0.037*"chance" + 0.033*"family" + 0.031*"reason" + 0.027*"treat" + '
  '0.025*"problem" + 0.022*"reasonable"'),
 (2,
  '0.464*"good" + 0.098*"benefit" + 0.065*"offer" + 0.047*"salary" + '
  '0.022*"positive" + 0.019*"thing" + 0.014*"hourly" + 0.012*"reputation" + '
  '0.010*"standard" + 0.009*"compare"'),
 (3,
  '0.073*"many" + 0.050*"give" + 0.045*"helpful" + 0.043*"able" + '
  '0.038*"available" + 0.036*"way" + 0.034*"bonus" + 0.030*"contact" + '
  '0.026*"hard" + 0.024*"choose"'),
 (4,
  '0.422*"great" + 0.085*"management" + 0.078*"team" + 0.057*"support" + '
  '0.034*"manager" + 0.023*"strong" + 0.023*"value" + 0.014*"ethic" + '
  '0.014*"plan" + 0.013*"website"'),
 (5,
  '0.136*"nice" + 0.098*"day" + 0.068*"office" + 0.

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt


Perplexity:  -6.34527232572905

Coherence Score:  0.41218922900431565


In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.429507 -0.155198       1        1  10.804657
3     -0.024948  0.023986       2        1   6.903247
4     -0.023306  0.034692       3        1   6.481502
2     -0.008814 -0.003386       4        1   6.159264
8     -0.045613  0.175412       5        1   6.087633
6     -0.056484  0.368968       6        1   5.790459
18    -0.005843  0.007188       7        1   5.734120
12    -0.035066  0.079662       8        1   5.678847
11    -0.010092  0.010279       9        1   4.984206
19     0.006451 -0.023074      10        1   4.964367
10     0.041814 -0.044502      11        1   4.605663
5      0.051788 -0.048992      12        1   4.280549
16     0.047245 -0.042889      13        1   4.132304
15     0.056620 -0.056476      14        1   4.083263
9      0.071516 -0.052789      15        1   3.521033
17     0.062341 -0.057927      16        1   3.428361
7      0.075249 -0.052588      17        1   3.276578
14     0.061710 -0.047976      18        1   3.263394
13     0.080040 -0.058919      19        1   3.228874
1      0.084899 -0.055473      20        1   2.591676, topic_info=            Term         Freq        Total Category  logprob  loglift
5           work  4523.000000  4523.000000  Default  30.0000  30.0000
14          good  2198.000000  2198.000000  Default  29.0000  29.0000
34         great  2103.000000  2103.000000  Default  28.0000  28.0000
53           pay  1787.000000  1787.000000  Default  27.0000  27.0000
46       company  1576.000000  1576.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
1022       agent    17.235620    17.955470  Topic20  -4.7504   3.6119
1270    majority    16.236613    16.956463  Topic20  -4.8101   3.6095
19         point    15.933046    16.652895  Topic20  -4.8290   3.6087
2184  eventually    15.732105    16.451955  Topic20  -4.8417   3.6081
775     grateful    14.916128    15.635978  Topic20  -4.8950   3.6057

[631 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
635       1  0.995097     ability
394       2  0.997793        able
795       7  0.988075  acceptable
728      11  0.988429      access
142       3  0.983325    activity
...     ...       ...         ...
999      14  0.982008   workplace
29       16  0.992892       world
2571      4  0.980452   worldwide
1849     17  0.972182       write
305      14  0.997644        year

[601 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 5, 3, 9, 7, 19, 13, 12, 20, 11, 6, 17, 16, 10, 18, 8, 15, 14, 2])

In [ ]:
``